![A car dashboard with lots of new technical features.](images/dashboard.jpg)

You're working for a well-known car manufacturer who is looking at implementing LLMs into vehicles to provide guidance to drivers. You've been asked to experiment with integrating car manuals with an LLM to create a context-aware chatbot. They hope that this context-aware LLM can be hooked up to a text-to-speech software to read the model's response aloud.

As a proof of concept, you'll integrate several pages from a car manual that contains car warning messages and their meanings and recommended actions. This particular manual, stored as an HTML file, `mg-zs-warning-messages.html`, is from an MG ZS, a compact SUV. Armed with your newfound knowledge of LLMs and LangChain, you'll implement Retrieval Augmented Generation (RAG) to create the context-aware chatbot.

## Before you start

In order to complete the project you will need to create a developer account with OpenAI and store your API key as a secure environment variable. Instructions for these steps are outlined below.

### Create a developer account with OpenAI

1. Go to the [API signup page](https://platform.openai.com/signup). 

2. Create your account (you'll need to provide your email address and your phone number).

3. Go to the [API keys page](https://platform.openai.com/account/api-keys). 

4. Create a new secret key.

<img src="images/openai-new-secret-key.png" width="200">

5. **Take a copy of it**. (If you lose it, delete the key and create a new one.)

### Add a payment method

OpenAI sometimes provides free credits for the API, but this can vary depending on geography. You may need to add debit/credit card details. 

**This project should cost much less than 1 US cents with `gpt-4o-mini` (but if you rerun tasks, you will be charged every time).**

1. Go to the [Payment Methods page](https://platform.openai.com/account/billing/payment-methods).

2. Click Add payment method.

<img src="images/openai-add-payment-method.png" width="200">

3. Fill in your card details.

### Add an environmental variable with your OpenAI key

1. In the workbook, click on "Environment," in the top toolbar and select "Environment variables".

2. Click "Add" to add environment variables.

3. In the "Name" field, type "OPENAI_API_KEY". In the "Value" field, paste in your secret key.

<img src="images/datalab-env-var-details.png" width="500">

4. Click "Create", then you'll see the following pop-up window. Click "Connect," then wait 5-10 seconds for the kernel to restart, or restart it manually in the Run menu.

<img src="images/connect-integ.png" width="500">

### Update to Python 3.10

Due to how frequently the libraries required for this project are updated, you'll need to update your environment to Python 3.10:

1. In the workbook, click on "Environment," in the top toolbar and select "Session details".

2. In the workbook language dropdown, select "Python 3.10".

3. Click "Confirm" and hit "Done" once the session is ready.

In [46]:
# Update your environment to Python 3.10 as described above before running this cell
import subprocess
import pkg_resources

def install_if_needed(package, version):
    '''Function to ensure that the libraries used are consistent to avoid errors.'''
    try:
        pkg = pkg_resources.get_distribution(package)
        if pkg.version != version:
            raise pkg_resources.VersionConflict(pkg, version)
    except (pkg_resources.DistributionNotFound, pkg_resources.VersionConflict):
        subprocess.check_call(["pip", "install", f"{package}=={version}"])

install_if_needed("langchain-core", "0.3.18")
install_if_needed("langchain-openai", "0.2.8")
install_if_needed("langchain-community", "0.3.7")
install_if_needed("unstructured", "0.14.4")
install_if_needed("langchain-chroma", "0.1.4")
install_if_needed("langchain-text-splitters", "0.3.2")

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_core-0.3.18-py3-none-any.whl.metadata (6.3 kB)
Using cached langchain_core-0.3.18-py3-none-any.whl (409 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.34
    Uninstalling langchain-core-0.3.34:
      Successfully uninstalled langchain-core-0.3.34


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.18 requires langchain-core<1.0.0,>=0.3.34, but you have langchain-core 0.3.18 which is incompatible.
langchain 0.3.18 requires langchain-text-splitters<1.0.0,>=0.3.6, but you have langchain-text-splitters 0.3.2 which is incompatible.
crewai 0.30.11 requires langchain<0.2.0,>=0.1.10, but you have langchain 0.3.18 which is incompatible.
embedchain 0.1.110 requires langchain<0.2.0,>=0.1.4, but you have langchain 0.3.18 which is incompatible.
embedchain 0.1.110 requires langchain-openai<0.2.0,>=0.1.7, but you have langchain-openai 0.2.8 which is incompatible.
langchain-cohere 0.1.5 requires langchain-core<0.3,>=0.1.42, but you have langchain-core 0.3.18 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_core-0.3.34-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.3.6-py3-none-any.whl.metadata (1.9 kB)
Using cached langchain_core-0.3.34-py3-none-any.whl (412 kB)
Using cached langchain_text_splitters-0.3.6-py3-none-any.whl (31 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.18
    Uninstalling langchain-core-0.3.18:
      Successfully uninstalled langchain-core-0.3.18
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.2
    Uninstalling langchain-text-splitters-0.3.2:
      Successfully uninstalled langchain-text-splitters-0.3.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
crewai 0.30.11 requires langchain<0.2.0,>=0.1.10, but you have langchain 0.3.18 which is incompatible.
embedchain 0.1.110 requires langchain<0.2.0,>=0.1.4, but you have langchain 0.3.18 which is incompatible.
embedchain 0.1.110 requires langchain-openai<0.2.0,>=0.1.7, but you have langchain-openai 0.2.8 which is incompatible.
langchain-cohere 0.1.5 requires langchain-core<0.3,>=0.1.42, but you have langchain-core 0.3.34 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_text_splitters-0.3.2-py3-none-any.whl.metadata (2.3 kB)
Using cached langchain_text_splitters-0.3.2-py3-none-any.whl (25 kB)
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.6
    Uninstalling langchain-text-splitters-0.3.6:
      Successfully uninstalled langchain-text-splitters-0.3.6


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.18 requires langchain-text-splitters<1.0.0,>=0.3.6, but you have langchain-text-splitters 0.3.2 which is incompatible.
crewai 0.30.11 requires langchain<0.2.0,>=0.1.10, but you have langchain 0.3.18 which is incompatible.
embedchain 0.1.110 requires langchain<0.2.0,>=0.1.4, but you have langchain 0.3.18 which is incompatible.
embedchain 0.1.110 requires langchain-openai<0.2.0,>=0.1.7, but you have langchain-openai 0.2.8 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [47]:
# Set your API key to a variable
import os
openai_api_key = os.environ["MY_VARIABLE_NAME"]

# Import the required packages
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_openai import OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

In [48]:
# Load the HTML as a LangChain document loader
loader = UnstructuredHTMLLoader(file_path="data/mg-zs-warning-messages.html")
car_docs = loader.load()

In [49]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(car_docs)

In [50]:
print(chunks)

[Document(metadata={'source': 'data/mg-zs-warning-messages.html'}, page_content='Warning Message Procedure Cruise Control Fault Indicates that the cruise control system has detected a fault. Please consult an MG Authorised Repairer as soon as possible. Active Speed Limiter Fault Indicates that the active speed limit system has detected a fault. Contact an MG Authorised Repairer as soon as possible. Engine Coolant Temperature High High engine coolant temperature could result in severe damage. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately. Engine Coolant Temperature Sensor Fault Indicates that the engine coolant temperature sensor has failed. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately.'), Document(metadata={'source': 'data/mg-zs-warning-messages.html'}, page_content='Warning Message Procedure Low Oil Pressure Indicates 

In [51]:
print([len(chunk.page_content)for chunk in chunks])

[782, 652, 613, 491, 604, 681, 633, 677, 950]


In [52]:
embedding_model = OpenAIEmbeddings(
    api_key= openai_api_key,
    model = "text-embedding-3-small"
)

In [53]:
vector_store = Chroma.from_documents(
    documents= chunks,
    embedding= embedding_model
)

In [54]:
retriever = vector_store.as_retriever()

In [55]:
prompt = ChatPromptTemplate.from_template(""" 
Use the following pieces of context to answer the question at the end. If you don't know the answer, say that you don't know. 
Context: {context} 
Question: {question}
Answer:
"""
)

In [56]:
llm = ChatOpenAI(model = "gpt-4o-mini", api_key=openai_api_key, temperature=0)

In [57]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

In [58]:
query = "The Gasoline Particular Filter Full warning has appeared. What does this mean and what should I do about it?"


In [59]:
answer = chain.invoke(query).content
print(answer)

The Gasoline Particular Filter Full warning indicates that the gasoline particular filter is full. You should consult an MG Authorised Repairer as soon as possible to address this issue.
